In [1]:
#kết nối với gg drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#đặt thêm đường dẫn tìm kiếm
import sys
sys.path.append('/content/gdrive/My Drive')

In [3]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.1 MB/s 


In [4]:
!unzip gdrive/MyDrive/MURA-v1.1.zip

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
   creating: MURA-v1.1/train/XR_SHOULDER/patient01061/
   creating: MURA-v1.1/train/XR_SHOULDER/patient01061/study1_positive/
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01061/study1_positive/image1.png  
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01061/study1_positive/image3.png  
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01061/study1_positive/image2.png  
   creating: MURA-v1.1/train/XR_SHOULDER/patient01016/
   creating: MURA-v1.1/train/XR_SHOULDER/patient01016/study1_positive/
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01016/study1_positive/image1.png  
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01016/study1_positive/image3.png  
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01016/study1_positive/image2.png  
   creating: MURA-v1.1/train/XR_SHOULDER/patient01182/
   creating: MURA-v1.1/train/XR_SHOULDER/patient01182/study1_positive/
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01182/study1_positi

In [5]:
import os
import pathlib
import random
import re
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.metrics import cohen_kappa_score
from sklearn.utils import shuffle

from utils import (
    clean_up,
    F1Score,
    inspect_df,
    plot_metrics,
)


pd.set_option("max_colwidth", None)
#plt.style.use("dark_background") 'default'

In [6]:
gpus = tf.config.experimental.list_physical_devices("GPU")

try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
except IndexError:
    print("Cannot memory-restrict the GPU, if no GPU exists in system. Ignore...")
    pass

In [7]:
SEED = 314

os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)
np.random.seed(SEED)

### Data Loading

In [8]:
DATASET_DIR = "/content/MURA-v1.1"

In [9]:
train_image_paths = pd.read_csv(
    os.path.join(DATASET_DIR, "train_image_paths.csv"), #"train_image_paths.csv"
    names=["image_path"],
    header=None,
    index_col=False,
)

inspect_df(train_image_paths)

shape: (36808, 1)


,image_path
0,MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image1.png
1,MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image2.png
2,MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image3.png
3,MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/image1.png
4,MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/image2.png


In [10]:
train_image_paths["image_path"] = train_image_paths["image_path"].map(
    lambda x: os.path.join("/content/", x)
)
train_image_paths["study_type"] = train_image_paths["image_path"].map(
    lambda x: x.split("/")[4]
)
train_image_paths["patient"] = train_image_paths["image_path"].map(
    lambda x: x.split("/")[5]
)
train_image_paths["study"] = train_image_paths["image_path"].map(
    lambda x: x.split("/")[6]
)
train_image_paths["study_path"] = train_image_paths["image_path"].map(
    lambda x: re.sub(r"image\d+.png", "", x)
)

In [11]:
inspect_df(train_image_paths)

shape: (36808, 5)


,image_path,study_type,patient,study,study_path
0,/content/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image1.png,XR_SHOULDER,patient00001,study1_positive,/content/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/
1,/content/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image2.png,XR_SHOULDER,patient00001,study1_positive,/content/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/
2,/content/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image3.png,XR_SHOULDER,patient00001,study1_positive,/content/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/
3,/content/MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/image1.png,XR_SHOULDER,patient00002,study1_positive,/content/MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/
4,/content/MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/image2.png,XR_SHOULDER,patient00002,study1_positive,/content/MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/


In [12]:
train_labeled_studies = pd.read_csv(
    os.path.join(DATASET_DIR, "train_labeled_studies.csv"), #"train_labeled_studies.csv"
    names=["study_path", "label"],
    header=None,
    index_col=False,
)

inspect_df(train_labeled_studies)

shape: (13457, 2)


,study_path,label
0,MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/,1
1,MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/,1
2,MURA-v1.1/train/XR_SHOULDER/patient00003/study1_positive/,1
3,MURA-v1.1/train/XR_SHOULDER/patient00004/study1_positive/,1
4,MURA-v1.1/train/XR_SHOULDER/patient00005/study1_positive/,1


In [13]:
train_labeled_studies["study_path"] = train_labeled_studies["study_path"].map(
    lambda x: os.path.join("/content/", x)
)
train_labeled_studies["label"] = train_labeled_studies["label"].map(str)

In [14]:
inspect_df(train_labeled_studies)

shape: (13457, 2)


,study_path,label
0,/content/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/,1
1,/content/MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/,1
2,/content/MURA-v1.1/train/XR_SHOULDER/patient00003/study1_positive/,1
3,/content/MURA-v1.1/train/XR_SHOULDER/patient00004/study1_positive/,1
4,/content/MURA-v1.1/train/XR_SHOULDER/patient00005/study1_positive/,1


In [15]:
ref_trainset = pd.merge(
    train_image_paths, train_labeled_studies, how="inner", on="study_path"
)

inspect_df(ref_trainset)

shape: (36808, 6)


,image_path,study_type,patient,study,study_path,label
0,/content/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image1.png,XR_SHOULDER,patient00001,study1_positive,/content/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/,1
1,/content/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image2.png,XR_SHOULDER,patient00001,study1_positive,/content/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/,1
2,/content/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image3.png,XR_SHOULDER,patient00001,study1_positive,/content/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/,1
3,/content/MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/image1.png,XR_SHOULDER,patient00002,study1_positive,/content/MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/,1
4,/content/MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/image2.png,XR_SHOULDER,patient00002,study1_positive,/content/MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/,1


#### Create a tensorflow input pipeline for the training data

In [16]:
IMAGE_WIDTH = IMAGE_HEIGHT = 360 #224
BATCH_SIZE = 8
#chỉnh lại learning rate
#We used an initial learning rate of 0.0001 that is decayed by a factor of 10 
#each time the validation loss plateaus after an epoch.
# thử giảm alpha sau 1 epoch

In [17]:
ref_trainse=ref_trainset

In [18]:
METRICS = [
    tf.keras.metrics.TruePositives(name="tp"),
    tf.keras.metrics.FalsePositives(name="fp"),
    tf.keras.metrics.TrueNegatives(name="tn"),
    tf.keras.metrics.FalseNegatives(name="fn"),
    tf.keras.metrics.BinaryAccuracy(name="binary_acc"),
    tf.keras.metrics.Precision(name="precision"),
    tf.keras.metrics.Recall(name="recall"),
    F1Score(name="f1_score"),
    tf.keras.metrics.AUC(name="roc_auc", curve="ROC"),
    tf.keras.metrics.AUC(name="pr_auc", curve="PR"),
    tfa.metrics.CohenKappa(name="cohen_kappa", num_classes=2),
]

In [19]:
loaistudy=['XR_ELBOW','XR_FINGER','XR_FOREARM','XR_HAND','XR_HUMERUS','XR_SHOULDER','XR_WRIST']
#loaistudy= ['allstudy']
for study in loaistudy:
  model = tf.keras.models.load_model(
    f"/content/gdrive/MyDrive/model/mobilenet {study}.h5",
    custom_objects={"F1Score": F1Score},
  )
  ref_trainset=ref_trainse[ref_trainse['study_type']==study]
  ref_trainset = shuffle(ref_trainset, random_state=SEED)
  training = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    horizontal_flip=True,
    fill_mode="constant", #thêm viền nếu cần
    cval=0.0,              # viền =0
    #validation_split=0.1, #0.2
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    #rescale=1./255,
    
  )
  validationset = training.flow_from_dataframe(
    dataframe=ref_trainset,
    x_col="image_path",
    y_col="label",
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    class_mode="binary",
    batch_size=BATCH_SIZE,
    seed=SEED,
    shuffle=False,
    #subset="validation",
      #color_mode="grayscale",
  )

  evaluation_metrics = model.evaluate(validationset, verbose=1)
  loss = evaluation_metrics.pop(0)
  print (study)
  print(f"val loss    : {loss}")
  for metric, score in zip(METRICS, evaluation_metrics):

    print(f"{metric.name + ' ' * (12 - len(metric.name))}: {score}")


Found 4931 validated image filenames belonging to 2 classes.
617/617 [==============================] - 150s 226ms/step - loss: 0.3225 - tp: 1587.0000 - fp: 216.0000 - tn: 2709.0000 - fn: 419.0000 - binary_acc: 0.8712 - precision: 0.8802 - recall: 0.7911 - f1_score: 0.8333 - roc_auc: 0.9296 - pr_auc: 0.9241 - cohen_kappa: 0.7289
XR_ELBOW
val loss    : 0.3225034177303314
tp          : 1587.0
fp          : 216.0
tn          : 2709.0
fn          : 419.0
binary_acc  : 0.8712228536605835
precision   : 0.880199670791626
recall      : 0.7911266088485718
f1_score    : 0.8332895636558533
roc_auc     : 0.9296166300773621
pr_auc      : 0.9240688681602478
cohen_kappa : 0.7288675308227539
Found 5106 validated image filenames belonging to 2 classes.
639/639 [==============================] - 148s 229ms/step - loss: 0.3605 - tp: 1628.0000 - fp: 479.0000 - tn: 2659.0000 - fn: 340.0000 - binary_acc: 0.8396 - precision: 0.7727 - recall: 0.8272 - f1_score: 0.7990 - roc_auc: 0.9211 - pr_auc: 0.8987 - cohe